In [1]:
import datetime as dt

from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *

In [2]:
rosters = retrieve_all_active_rosters(file_name = None)

In [5]:
rosters

[{'team_name': 'Angels',
  'team_roster': [('Jo Adell', '666176'),
   ('Jaime Barria', '642545'),
   ('Archie Bradley', '605151'),
   ('Cooper Criswell', '681867'),
   ('Brendon Davis', '663478'),
   ('Reid Detmers', '672282'),
   ('Jhonathan Diaz', '646242'),
   ('Matt Duffy', '622110'),
   ('David Fletcher', '664058'),
   ('Jimmy Herget', '623474'),
   ('Raisel Iglesias', '628452'),
   ('Janson Junk', '676083'),
   ('Michael Lorenzen', '547179'),
   ('Aaron Loup', '571901'),
   ('Brandon Marsh', '669016'),
   ('Jose Marte', '665896'),
   ('Mike Mayers', '594577'),
   ('Jack Mayfield', '608686'),
   ('Shohei Ohtani', '660271'),
   ('Oliver Ortega', '661383'),
   ('Elvis Peguero', '665625'),
   ('Jose Quijada', '650671'),
   ('Anthony Rendon', '543685'),
   ('Luis Rengifo', '650859'),
   ('Jose Rojas', '670351'),
   ('Patrick Sandoval', '663776'),
   ('Max Stassi', '545358'),
   ('Jose Suarez', '660761'),
   ('Kurt Suzuki', '435559'),
   ('Noah Syndergaard', '592789'),
   ('Ryan Tepera